In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline

/home/sean/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 1. 训练

 ## 1. 1数据整理
读入数据，观察得到数据没有缺失值。数据主要存在两个问题，第一个就是没有下雨的RAINFOW值为NR，第二个就是存在大量的字符型数字，不利于将来处理，因此首先要解决这两个问题

In [2]:
data = pd.read_csv('train.csv', encoding='big5',engine='python')
data = data.iloc[:, range(2, data.shape[1])]
test = pd.read_csv('test.csv', encoding='big5', header=None)

In [3]:
data.columns = ['factor', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [4]:
data.head()

,factor,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,AMB_TEMP,14,14,14,13,12,12,12,12,15,...,22,22,21,19,17,16,15,15,15,15
1,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


发现PM2.5浓度的一些异常值

In [5]:
(data[data.factor == 'PM2.5'] < 0).sum()

factor    240
0         240
1         240
2         240
3         240
4         240
5         240
6         240
7         240
8         240
9         240
10        240
11        240
12        240
13        240
14        240
15        240
16        240
17        240
18        240
19        240
20        240
21        240
22        240
23        240
dtype: int64

In [6]:
test.head()

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,15,14,14,13,13,13,13,13,12
1,id_0,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,id_0,CO,0.36,0.35,0.34,0.33,0.33,0.34,0.34,0.37,0.42
3,id_0,NMHC,0.11,0.09,0.09,0.1,0.1,0.1,0.1,0.11,0.12
4,id_0,NO,0.6,0.4,0.3,0.3,0.3,0.7,0.8,0.8,0.9


`step1` 首先将所有RAINFALL中的NR值替换成0， 这里也可以考虑替换成-1， 最后可以修改下测试效果

In [7]:
NR = -1
test = test.replace('NR', NR)
data = data.replace('NR', NR)

`step2` 接下来将所有字符串型数字的格式改成浮点数，方便后面的操作

In [8]:
for i in np.arange(0, 24):
    str_i = str(i)
    data[str_i] = data[str_i].astype('float')

for i in np.arange(2, 11):
    test[i] = test[i].astype('float')

In [9]:
data.head()

,factor,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,AMB_TEMP,14.00,14.00,14.00,13.00,12.00,12.00,12.00,12.00,15.00,...,22.00,22.00,21.00,19.00,17.00,16.00,15.00,15.00,15.00,15.00
1,CH4,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,...,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80
2,CO,0.51,0.41,0.39,0.37,0.35,0.30,0.37,0.47,0.78,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,NMHC,0.20,0.15,0.13,0.12,0.11,0.06,0.10,0.13,0.26,...,0.10,0.13,0.14,0.23,0.18,0.12,0.10,0.09,0.10,0.08
4,NO,0.90,0.60,0.50,1.70,1.80,1.50,1.90,2.20,6.60,...,2.50,2.20,2.50,2.30,2.10,1.90,1.50,1.60,1.80,1.50


In [10]:
test.head()

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,15.00,14.00,14.00,13.00,13.00,13.00,13.00,13.00,12.00
1,id_0,CH4,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80
2,id_0,CO,0.36,0.35,0.34,0.33,0.33,0.34,0.34,0.37,0.42
3,id_0,NMHC,0.11,0.09,0.09,0.10,0.10,0.10,0.10,0.11,0.12
4,id_0,NO,0.60,0.40,0.30,0.30,0.30,0.70,0.80,0.80,0.90


## 1.2 相关系数
将数据堆积成列， 计算其相关系数， 只取相关系数高一点的前几项

In [11]:
words = np.array(data.factor)[:18]
arrange_data = pd.DataFrame(columns=words)

In [12]:
for i in np.arange(int((data.shape[0]) / 18)):
    select = data.iloc[range(i*18, i*18 + 18), :]
    value = (select.iloc[:,1:]).T
    value.columns = words
    arrange_data = arrange_data.append(value, ignore_index=True)

In [13]:
index_wa_pm2_5 = arrange_data[arrange_data['PM2.5'] < 0].index

In [14]:
index_wa_pm2_5

Int64Index([ 469, 1450, 1618, 1791, 2024, 2025, 2026, 2027, 2028, 2029, 2041,
            2042, 2043, 2049, 2059, 2060, 2061, 2065, 2066, 2067, 2068, 2069,
            2070, 2071, 2072, 2073, 2074, 2075, 2218, 2219, 3061, 3062, 5177,
            5178, 5179, 5180, 5181, 5182, 5183, 5184, 5185, 5186, 5187, 5188,
            5189, 5190, 5191, 5192, 5193, 5194, 5460, 5461],
           dtype='int64')

In [15]:
arrange_data.loc[index_wa_pm2_5, 'PM2.5'] = arrange_data['PM2.5'].mean()

In [16]:
corr = abs(arrange_data.corr()['PM2.5']).sort_values()
print(corr)
##这里先考虑相关系数d大于0.3
corr_list = list(corr[corr > 0.2].index)

AMB_TEMP      0.019570
NO            0.029595
WS_HR         0.051902
RAINFALL      0.072021
WIND_SPEED    0.086174
WIND_DIREC    0.156301
WD_HR         0.184678
CH4           0.252224
RH            0.260336
CO            0.284343
NMHC          0.292115
THC           0.351392
O3            0.359572
SO2           0.369728
NOx           0.376775
NO2           0.450664
PM10          0.771636
PM2.5         1.000000
Name: PM2.5, dtype: float64


## 1.3 训练模型
计算X_train, Y_train, 调用sckit_learn进行训练

In [17]:
arrange_data = arrange_data.loc[:, corr_list]

将其所有向量正规化

In [18]:
for i in np.arange(arrange_data.shape[1]):
    arrange_data.iloc[:,i] = (arrange_data.iloc[:,i] - np.mean(arrange_data.iloc[:,i])) / np.std(arrange_data.iloc[:,i])

In [19]:
arrange_data.head()

,CH4,RH,CO,NMHC,THC,O3,SO2,NOx,NO2,PM10,PM2.5
0,0.779246,0.282244,0.375952,0.569336,0.881886,-0.850475,-0.530127,0.627241,0.949409,0.506895,0.265289
1,0.779246,-0.391399,0.066875,0.091488,0.881886,-0.101887,-0.420042,-0.323069,-0.149667,0.278062,1.052062
2,0.779246,-0.466248,0.005060,-0.099651,0.881886,-0.262298,-0.585169,-0.468255,-0.311295,0.201785,0.870499
3,0.779246,0.057696,-0.056755,-0.195221,0.331901,-0.476181,-0.640211,-0.481454,-0.521413,-0.294020,0.809978
4,0.779246,-0.092002,-0.118571,-0.290791,0.331901,-0.422710,-0.475084,-0.494653,-0.537576,-0.675408,0.567894


In [20]:
X_train = []
Y_train = []

In [21]:
for i in np.arange(int((arrange_data.shape[0]) / 24 / 20)):
    current_month = arrange_data.iloc[np.arange(i * 24 * 20, (i + 1)*24*20), :]
    for j in np.arange(len(current_month) - 9):
        x = current_month.iloc[range(j, j + 9), :]
        y = current_month.iloc[j+9, :]
        X_train.append(list(np.array(x).flatten()))
        Y_train.append([y['PM2.5']])

In [22]:
model = LinearRegression(normalize=True, fit_intercept=True)
np.average(cross_val_score(model, X_train, Y_train, cv=10))

0.8272456005510126

In [23]:
model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

整理测试数据集

# 2.测试

## 2.1 整理test_data 

In [24]:
arrange_test_data = pd.DataFrame(columns=words)

for i in np.arange(int((test.shape[0]) / 18)):
    select = test.iloc[range(i*18, i*18 + 18), :]
    value = (select.iloc[:,2:]).T
    value.columns = words
    arrange_test_data = arrange_test_data.append(value)
    
arrange_test_data = arrange_test_data.loc[:, corr_list]

In [25]:
PM_mean = np.mean(arrange_test_data['PM2.5'])
PM_std = np.std(arrange_test_data['PM2.5'])

In [26]:
for i in np.arange(arrange_test_data.shape[1]):
    arrange_test_data.iloc[:,i] = (arrange_test_data.iloc[:,i] - np.mean(arrange_test_data.iloc[:,i])) / np.std(arrange_test_data.iloc[:,i])

In [27]:
arrange_test_data.head()

,CH4,RH,CO,NMHC,THC,O3,SO2,NOx,NO2,PM10,PM2.5
2,0.926332,0.231311,-0.171431,-0.430157,0.193980,0.113023,-1.131638,-0.425977,-0.281254,0.007036,-0.027917
3,0.926332,-0.059257,-0.215788,-0.625027,-0.425838,0.495740,-1.131638,-0.721389,-0.611945,0.007036,-0.680324
4,0.926332,-0.059257,-0.260145,-0.625027,-0.425838,0.543580,-1.131638,-0.856786,-0.762259,-0.604515,-0.167719
5,0.926332,0.086027,-0.304501,-0.527592,0.193980,0.495740,-0.905441,-0.918330,-0.822385,-0.329317,0.065284
6,0.926332,0.158669,-0.304501,-0.527592,0.193980,0.495740,-0.961990,-0.930639,-0.852448,-0.512782,0.624490


## 2.2 计算Yhat 

In [28]:
X_test = []
for i in np.arange(int((arrange_test_data.shape[0]) / 9)):
    current_day = arrange_test_data.iloc[np.arange(i * 9, (i + 1)* 9), :]
    X_test.append(list(np.array(current_day).flatten()))
    
Yhat = ((model.predict(X_test)).flatten()) * PM_std + PM_mean
real=pd.read_csv('https://ntumlta.github.io/2017fall-ml-hw1/ans.csv')
Y = np.array(real.value)

In [30]:
abs(Yhat - Y).sum() / len(Y)

4.95306759107929